In [2]:
import pandas as pd
import json
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import json
import numpy as np
%matplotlib inline

In [16]:
def read_csv(fname):
    #f = lambda x : json.loads(x.replace("'","\""))
    #df = pd.read_csv(fname, converters={'sensor_type': f}, index_col=0)
    df = pd.read_csv(fname, index_col=0)
    return df

In [7]:
def save_graph(plot, fname='saved.png'):
    fig = plot.get_figure()
    fig.savefig(fname)

In [56]:
keys = {'all': ['ma_window', 'sensor_type', 'n_record', 'n_train', 'ms_interval'],
        'kNN': ['n_neighbors', 'algorithm'],
        'k-means': ['n_clusters', 'max_iter'],
        'NN': ['num_of_hidden_nodes', 'num_of_training_epochs', 'size_of_mini_batch', 'learning_rate'],
       }

def df_values(_df):
    for c in keys['all']:
        if c == 'auc': continue
        print("%-40s%s" % (c, _df[c].unique()))

    print('--- kNN')
    for c in keys['kNN']:
        if c == 'auc': continue
        print("%-40s%s" % (c, _df[_df['type']=='kNN'][c].unique()))
    
    print('--- k-means')
    for c in keys['k-means']:
        if c == 'auc': continue
        print("%-40s%s" % (c, _df[_df['type']=='k-means'][c].unique()))
    
    print('--- NN')
    for c in keys['NN']:
        if c == 'auc': continue
        print("%-40s%s" % (c, _df[_df['type']=='NN'][c].unique()))

In [44]:
csv_file = 'test_record-20170425.csv'
df = read_csv(csv_file)
df.sort_values(by=['auc'], ascending=False).head()

df1 = df[
    (df['n_record'] == 1) & 
    (df['ms_interval'] == 20) &
    (df['sensor_type'].apply(lambda x: len(x) == 2))
]
display(df1)

def show_graph(df, x_title, y_title):
    x_title = 'n_train'
    y_title = 'auc'
    
    df1 = df[df['type'] == 'k-means']
    df2 = df[df['type'] == 'kNN']
    df3 = df[df['type'] == 'rssi_based']

    x = df1[x_title].values
    y1 = df1[y_title].values
    y2 = df2[y_title].values
    y3 = df3[y_title].values
    print(y3)

    df = pd.DataFrame(np.array([x, y1, y2, y3]).T, columns=['n_train', 'k-means', 'kNN', 'rssi_based'])
    display(df)
    df.plot.bar(x='n_train', y=['k-means', 'kNN', 'rssi_based'], ylim=[.0, 1.2])

#show_graph(df1, 'n_train', 'auc')

,algorithm,auc,ma_window,max_iter,model_idx,ms_interval,n_clusters,n_neighbors,n_record,n_train,sensor_type,type


In [57]:
#csv_file = 'test_record-20170425.csv'
csv_file = 'test_record_02_11_01__^02_11__^02_1[23].csv.bak_20170428_0916'
df = read_csv(csv_file)

_d = df[
    (df['n_record'] == 1) & 
    (df['ms_interval'] == 20) &
    (df['sensor_type'].apply(lambda x: 'gyro.rotationRate' in x))
]
_dknn = _d[_d['type'] == 'kNN']

#display(_dknn)

df_values(_d)

def show_graph(df, x_title, y_title):
    x_title = 'n_train'
    y_title = 'auc'
    
    df1 = df[df['type'] == 'k-means']
    df2 = df[df['type'] == 'kNN']
    df3 = df[df['type'] == 'rssi_based']

    x = df1[x_title].values
    y1 = df1[y_title].values
    y2 = df2[y_title].values
    y3 = df3[y_title].values
    print(y3)

    df = pd.DataFrame(np.array([x, y1, y2, y3]).T, columns=['n_train', 'k-means', 'kNN', 'rssi_based'])
    display(df)
    df.plot.bar(x='n_train', y=['k-means', 'kNN', 'rssi_based'], ylim=[.0, 1.2])

#show_graph(df1, 'n_train', 'auc')

ma_window                               [1]
sensor_type                             [ '["rssi.a", "rssi.b", ["gyro.rotationRate.x", "gyro.rotationRate.y", "gyro.rotationRate.z"]]']
n_record                                [1]
n_train                                 [  1   5  10  20 100]
ms_interval                             [20]
--- kNN
n_neighbors                             [ 1.  3.  5.  7.  9.]
algorithm                               ['ball_tree']
--- k-means
n_clusters                              [ 1.  3.  5.  7.  9.]
max_iter                                [ 3.]
--- NN
num_of_hidden_nodes                     [  4.   8.  16.  32.]
num_of_training_epochs                  [ 100.]
size_of_mini_batch                      [ 10.]
learning_rate                           [ 0.02]
